In [1]:
from transformers import (
     Blip2QFormerConfig,
     Blip2QFormerModel,
 )
import sys

import torch
from transformers import AutoModelForCausalLM

from model.memory_bank_ours.models import VLChatProcessor,apply_memory_bank

from model.deepseek_vl.utils.io import load_pil_images
from torch import nn
from einops import rearrange
from torchsummary import summary

/data/Users/xyq/developer/happy_code/.venv/.dpo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-19 13:21:12,205] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  please install triton==1.0.0 if you want to use sparse attention
Python version is above 3.10, patching the collections module.
Python version is above 3.10, patching the collections module.


In [2]:
model_path = "checkpoints/memory_bank_1.3b_based_on_sft/2024-07-18-22-20"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda:3", torch.float16).eval()
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
vl_chat_processor.num_image_tokens = model.model_config.qformer_config.num_query_tokens

In [4]:
conversation = [

      {
        "role": "User",
        "type": "chosen",
        "content": "Current Goal: chop a tree. Predict the fice action. <image_placeholder><a><action></a><image_placeholder>",
        "images": [
          "/data/Users/xyq/developer/happy_code/data/action_dpo/v1/mc_dataset_test/chop_a_tree_3/chop_a_tree_3_frame_0.jpg",
          "/data/Users/xyq/developer/happy_code/data/action_dpo/v1/mc_dataset_test/chop_a_tree_3/chop_a_tree_3_frame_3.jpg"
        ]
      },{
          "role": "Assistant",
          "content": ""
      }
      ]

In [14]:
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(conversations=conversation, images=pil_images, force_batchify=True).to(
    model.device, torch.float16
)

In [15]:
prepare_inputs["images_seq_mask"].shape

torch.Size([1, 126])

In [17]:
input_embeds = model.prepare_inputs_embeds(**prepare_inputs)

outputs = model.language_model.generate(
    inputs_embeds=input_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
answer

' <a><attack></a><a><attack></a><a><attack></a><a><attack></a><｜end▁of▁sentence｜>'